# Thin structures

In this notebook, we consider a cylindrical shell as 3D domain, where the thickness is significantly small. This leads to shear-locking for the standard formulation of linear elasticity. We also test the TDNNS method (briefly introduced in the next notebook), which is shear-locking free.

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

A cylindrical shell domain, where both ends are rigidity fixed (only the normal displacement is allowed to move there). By prescribing a periodic volume force, we can exploit the geometry's symmetry to mesh only one eight of the original geometry and then prescribe symmetry boundary conditions on the arising boundaries.

In [ ]:
# thickness, radius and length of the cylinder
t  = 3
Ri = 300
L  = 600

# Young's modulus and Poisson's ratio
E  = 3e6
nu = 0.3

def GenerateMesh(maxh=200, order=1):
    c1 = Cylinder((0,0,0), (0,0,1), r=Ri+t, h=L, mantle="outer")
    c2 = Cylinder((0,0,0), (0,0,1), r=Ri, h=L, mantle="inner")

    shell = (c1-c2)*Box((-L+t,0,0), (0,L+t,L))
    shell.faces.Min(Z).name = "back"
    shell.faces.Max(Z).name = "front"
    shell.faces.Min(Y).name = "bottom"
    shell.faces.Max(X).name = "right"

    trf = gp_GTrsf(((Ri + t)/Ri,0,0,0,(Ri + t)/Ri,0,0,0,1), (0,0,0))
    shell.faces["inner"].Identify(shell.faces["outer"], "cs",
                            IdentificationType.CLOSESURFACES,
                            trf)

    mesh = Mesh(OCCGeometry(shell).GenerateMesh(maxh=maxh, quad_dominated=True)).Curve(order)

    return mesh

mesh = GenerateMesh(maxh=200, order=2)
Draw (mesh);

In [ ]:
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

force = CF( (0,-10,0) )

We start with standard discretization of linear elasticity with Lagrange elements.

In [ ]:
def Solve(mesh, order=1):
    fes = VectorH1(mesh, order=order, dirichletx="back|right", dirichlety="back|bottom",\
                dirichletz="front")
    u,v = fes.TnT()
    
    a = BilinearForm(fes, symmetric=True)
    a += (2*mu*InnerProduct(Sym(Grad(u)),Sym(Grad(v)))+lam*div(u)*div(v))*dx

    f = LinearForm(fes)
    f += force*v*dx

    gfu = GridFunction(fes)
    a.Assemble()
    f.Assemble()

    gfu.vec.data = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")*f.vec
    return gfu


# try out:
# order=1; maxh=200
# order=1; maxh=100
# order=1; maxh=50
# order=1; maxh=25
# order=2; maxh=200
# order=2; maxh=100
# order=2; maxh=50
# order=3; maxh=200
# order=3; maxh=100
# order=3; maxh=50
order = 1

mesh = GenerateMesh(maxh=200, order=order)
with TaskManager():
    gfu = Solve(mesh, order)
    
Draw(gfu, mesh, deformation=True);

Next, we use the TDNNS method.

In [ ]:
def SolveTDNNS(mesh, order=1):
    V = HCurl(mesh, order=order, dirichlet="back")
    S = HDivDiv(mesh,order=order,dirichlet="back|outer|inner")
    fes = V*S
    (u,sigma),(v,tau) = fes.TnT()
    
    n = specialcf.normal(3)
    def tang(vec): return vec - (vec*n)*n
    
    a = BilinearForm(fes, symmetric=True)
    # small regularization -E*1e-10*u*v to allow for sparsecholesky solver
    a += (1/E*InnerProduct((1+nu)*Deviator(sigma)+(1-2*nu)/2*Trace(sigma)*Id(3),tau)\
        + div(sigma)*v + div(tau)*u -E*1e-10*u*v)*dx
    a += -(tang(sigma*n)*v + tang(tau*n)*u)*dx(element_boundary=True)

    f = LinearForm(fes)
    f += -force*v*dx

    gfsol = GridFunction(fes)
    a.Assemble()
    f.Assemble()

    gfsol.vec.data = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")*f.vec
    gfu, _ = gfsol.components
    return gfu


# try out:
# order=1; maxh=200
# order=1; maxh=100
# order=1; maxh=50
# order=2; maxh=200
# order=2; maxh=100
# order=3; maxh=200
# order=3; maxh=100
order = 1

mesh = GenerateMesh(maxh=200, order=order)
with TaskManager():
    gfu = SolveTDNNS(mesh, order)

# the BoundaryFromVolumeCF is needed to draw the solution correctly at the boundary
# remember that H(curl) has only a well-defined tangential trace
Draw(BoundaryFromVolumeCF(gfu), mesh, deformation=True);